# CASE – ABSENTEÍSMO
## 02 – Exploração da Base de HeadCount

Objetivo:
- Consolidar os 3 arquivos de headcount.
- Verificar inconsistências.
- Entender estrutura e qualidade dos dados.

In [47]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

In [48]:
path = "../banco_de_headcount/*.csv"

files = glob.glob(path)

df_headcount = pd.concat(
    [pd.read_csv(file, sep=";") for file in files],
    ignore_index=True
)

print("Total de registros:", df_headcount.shape[0])
df_headcount.head()

Total de registros: 13600


,id_funcionario,ano_mes,ativo_no_mes,uf,unidade,area,cargo,regime,turno,escala,dias_uteis_mes,base_horas_mes
0,FUNC0001,2025-01,1,PI,Filial Norte,Operações,Analista,CLT,Diurno,5x2,22,177
1,FUNC0001,2025-02,1,PI,Filial Norte,Operações,Analista,CLT,Diurno,5x2,23,183
2,FUNC0001,2025-03,1,PI,Filial Norte,Operações,Analista,CLT,Diurno,5x2,19,152
3,FUNC0001,2025-04,1,PI,Filial Norte,Operações,Analista,CLT,Diurno,5x2,21,167
4,FUNC0001,2025-05,1,PI,Filial Norte,Operações,Analista,CLT,Diurno,5x2,20,161


In [49]:
df_headcount.groupby(["id_funcionario", "ano_mes"]).size().sort_values(ascending=False)


id_funcionario  ano_mes
FUNC0002        2025-12    2
FUNC0849        2025-09    2
FUNC0002        2025-11    2
FUNC0001        2025-09    2
FUNC0850        2025-12    2
                          ..
FUNC0848        2025-07    1
                2025-06    1
                2025-05    1
FUNC0001        2025-04    1
                2025-03    1
Length: 10200, dtype: int64

In [50]:
df_headcount.shape


(13600, 12)

In [51]:
# verificar a distribuição de registros por mês
df_headcount["ano_mes"].value_counts()


ano_mes
2025-09    1700
2025-10    1700
2025-11    1700
2025-12    1700
2025-04     850
2025-03     850
2025-02     850
2025-01     850
2025-08     850
2025-07     850
2025-06     850
2025-05     850
Name: count, dtype: int64

In [52]:
# Verificando se existe funcionário com mais de um registro por mês

df_headcount.groupby(["id_funcionario", "ano_mes"]).size().value_counts()


1    6800
2    3400
Name: count, dtype: int64

In [53]:
# Manter maior base_horas_mes por funcionário/mês

# df_headcount = (
#     df_headcount
#     .sort_values("base_horas_mes", ascending=False)
#     .drop_duplicates(subset=["id_funcionario", "ano_mes"], keep="first")
# )

# df_headcount.head()

# 1️⃣ Priorizar registros ativos
df_headcount = (
    df_headcount
    .sort_values(by=["id_funcionario", "ano_mes", "ativo_no_mes"], ascending=[True, True, False])
)

# 2️⃣ Manter apenas o primeiro por funcionário/mês
df_headcount = df_headcount.drop_duplicates(subset=["id_funcionario", "ano_mes"], keep="first")


In [54]:
# Verificando se existe funcionário com mais de um registro por mês
df_headcount.groupby(["id_funcionario", "ano_mes"]).size().value_counts()

1    10200
Name: count, dtype: int64

In [55]:
# Manter apenas ativo_no_mes = 1

df_headcount = df_headcount[df_headcount["ativo_no_mes"] == 1]

In [56]:
# Verificando se existe funcionário com base_horas_mes = 0
df_headcount[df_headcount["base_horas_mes"] == 0]

,id_funcionario,ano_mes,ativo_no_mes,uf,unidade,area,cargo,regime,turno,escala,dias_uteis_mes,base_horas_mes


In [57]:
# Verificando se existe funcionário com base_horas_mes = null
df_headcount[df_headcount["base_horas_mes"].isnull()]


,id_funcionario,ano_mes,ativo_no_mes,uf,unidade,area,cargo,regime,turno,escala,dias_uteis_mes,base_horas_mes


In [58]:
# Total de funcionários por mês
df_headcount["ano_mes"].value_counts().sort_index()


ano_mes
2025-01    778
2025-02    783
2025-03    789
2025-04    796
2025-05    808
2025-06    812
2025-07    817
2025-08    815
2025-09    817
2025-10    817
2025-11    817
2025-12    816
Name: count, dtype: int64

In [60]:
# Verificando a quantidade de registros por funcionário/mês com ativo_no_mes = 1
df_headcount.groupby(["id_funcionario", "ano_mes"]).size().value_counts()


1    9665
Name: count, dtype: int64


# 📌 Conclusões da Base de Headcount

- 13.600 registros na base consolidada (sem tratamento).
    - 6.800 com 1 registro.
    - 3.400 com 2 registros.
- Não existem base_horas_mes nulas.
- Não existem base_horas_mes iguais a zero.
- Registros em duplicidade no período de set/25 a dez/25.
- Alguns funcionários possuíam mais de um registro no mesmo mês.
- Em determinados casos havia registros com ativo e inativo no mesmo mês.
- Após tratamento:
    - Foi priorizado ativo_no_mes = 1.
    - Mantido apenas um registro por funcionário/mês.
    - Eliminadas duplicidades.
    - 9.665 registros por funcionário/mês com ativo_no_mes = 1.